In [31]:
# Loading Packages
import numpy as np
import scanpy as sc
import decoupler as dc
import pandas as pd
import os
import subprocess
from pathlib import Path

In [33]:
# set root 
phosphoproteome_ROOT = Path("/Users/ogawaakane/Yugi_lab_project/phosphoproteome")
phosphoproteome_ROOT

PosixPath('/Users/ogawaakane/Yugi_lab_project/phosphoproteome')

In [ ]:
# intensity of each phospho-site
intensity_per_psite = pd.read_csv(phosphoproteome_ROOT / "data" / "processed" / "intensity_per_psite.csv")
intensity_per_psite

,psite,sample,intensity,time,rep
0,AAGAB_S310,PK23030-1,14.942861,0,A
1,AAGAB_S310,PK23030-2,15.041506,0,B
2,AAGAB_S310,PK23030-3,15.187524,0,C
3,AAGAB_S310,PK23030-4,15.213950,5,A
4,AAGAB_S310,PK23030-5,15.201529,5,B
...,...,...,...,...,...
729955,ZZZ3_T160,PK23030-17,NaN,120,B
729956,ZZZ3_T160,PK23030-18,NaN,120,C
729957,ZZZ3_T160,PK23030-19,NaN,240,A
729958,ZZZ3_T160,PK23030-20,NaN,240,B


In [6]:
# data shaping
# 同じ時点のreplicateを平均化
expr = (
    intensity_per_psite
    .groupby(["psite", "time"], as_index=False)["intensity"]
    .mean()
    .pivot(index="psite", columns="time", values="intensity")
    # これを外せばpsite が行名（インデックス）の数値マトリックスになる
    #.reset_index() 
)
gene_names = intensity_per_psite["psite"].unique()

# 発現データをz score化する
expr_z = expr.sub(expr.mean(axis=1), axis=0).div(expr.std(axis=1), axis=0) # (値 - 行平均) / 行標準偏差
expr_z = expr_z.dropna(how="any") # NaNを含む行を削除

expr_z.head()

time,0,5,15,30,60,120,240
psite,,,,,,,
AAGAB_S310,-0.561163,0.480397,0.118534,0.935804,-1.556747,-0.701143,1.284319
AAGAB_S311,0.981355,0.409299,0.552500,0.569132,-1.438372,-1.429899,0.355985
AAK1_S20,0.412705,1.072255,0.780696,-1.185925,-1.376875,-0.453137,0.750281
AAK1_S21,0.937020,1.363256,0.608778,-0.167256,-1.300442,-0.980523,-0.460833
AAK1_S624,-0.117705,1.115899,1.176471,-0.730903,-1.562022,-0.374291,0.492550


In [7]:
# obs（サンプル情報）を作成
obs = pd.DataFrame(index=expr_z.columns)  # index: time 0, 1, 2, ...
obs.index.name = "sample"  # サンプル名（ここでは時間）
obs["time"] = obs.index.astype(int)

# expr_z は psite を行、time を列に持つデータフレーム
X = expr_z.T.values  # AnnData の構造に合わせるために転置

# var（遺伝子情報）を作成
var = pd.DataFrame(index=expr_z.index)  # index: psite
var.index.name = "psite"

# AnnData 作成
adata = sc.AnnData(X=X, obs=obs, var=var)

/Users/ogawaakane/Library/Python/3.9/lib/python/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [42]:
# Import net
## 現状pythonでkinase-substrate networkを取得するdecoupleRの関数は公開されていないため、Rで実行する
subprocess.run(
    ["Rscript", str(phosphoproteome_ROOT / "scripts/R/get_omnipath_ptm.R"),
     str(phosphoproteome_ROOT)], 
    cwd=phosphoproteome_ROOT
)
omnipath_ptm_path = phosphoproteome_ROOT / "data" / "raw" / "omnipath_ptm_250813.csv"
omnipath_ptm = pd.read_csv(omnipath_ptm_path)
omnipath_ptm


次のパッケージを付け加えます: ‘dplyr’

以下のオブジェクトは ‘package:stats’ からマスクされています:

    filter, lag

以下のオブジェクトは ‘package:base’ からマスクされています:

    intersect, setdiff, setequal, union

[2025-08-27 16:15:47] [SUCCESS] [OmnipathR] Loaded 391929 annotation records from cache.
[2025-08-27 16:15:47] [SUCCESS] [OmnipathR] Loaded 43269 enzyme-substrate relationships from cache.
[2025-08-27 16:15:48] [SUCCESS] [OmnipathR] Loaded 81529 interactions from cache.


,target,source,weight,likelihood
0,ITGB2_S745,PRKCH,1,1
1,ITGB2_T758,PRKCH,1,1
2,NPM1_S227,PRKCH,1,1
3,NPM1_T234,PRKCH,1,1
4,GSTP1_S43,PRKCH,1,1
...,...,...,...,...
30811,MAPK9_Y185,MAP3K4,1,1
30812,MAPK9_T183,MAP3K4,1,1
30813,MAPK10_Y223,MAP3K4,1,1
30814,MAPK10_T221,MAP3K4,1,1


In [19]:
# ===== ULM スコア計算 =====
dc.run_ulm(
    mat=adata,
    net=omnipath_ptm,
    use_raw=False,     # X を使う
    verbose=True 
)

Running ulm on mat with 7 samples and 25789 targets for 182 sources.


In [22]:
# ===== スコア（アクティビティ）取得 =====
score = dc.get_acts(adata, obsm_key='ulm_estimate')
score_df = score.to_df()

In [44]:
# キナーゼ活性推定の結果を保存
decoupleR_kin = score_df.T
file_path = phosphoproteome_ROOT / "results" / "tables" / "decoupleR_kin.csv"
decoupleR_kin.to_csv(file_path, index=True)
decoupleR_kin

sample,0,5,15,30,60,120,240
ABL1,-0.426737,-0.047711,-0.497784,-0.077026,0.542385,-0.323489,0.823894
AKT1,2.470201,0.967131,-2.141115,2.082965,1.142345,-1.937539,-3.164726
AKT2,3.303248,1.604716,-0.886623,0.453115,-0.323548,-1.933987,-3.264504
AKT3,0.848391,1.252373,-1.237064,0.038970,0.407253,-1.127549,-0.649992
ARAF,0.335499,-0.123778,-0.343865,-0.482889,0.595567,-0.777102,0.501792
...,...,...,...,...,...,...,...
VRK1,-2.888578,-1.995531,0.116989,0.370220,1.805935,2.358894,1.535608
VRK2,-2.629191,-1.137611,1.233398,0.116312,0.747887,1.730181,1.181876
WEE1,1.359460,0.731731,0.885033,-0.864500,-0.863791,-0.152395,-1.476892
WNK1,0.319455,0.775506,1.398410,-0.026572,0.774371,-1.078007,-1.795170


In [43]:
# draw lineplots of kinases
subprocess.run(
    ["Rscript", str(phosphoproteome_ROOT / "scripts/R/kin_lineplot.R"),
     str(phosphoproteome_ROOT)], 
    cwd=phosphoproteome_ROOT
)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
警告メッセージ:
1: パッケージ ‘ggplot2’ はバージョン 4.4.1 の R の下で造られました 
2: パッケージ ‘purrr’ はバージョン 4.4.1 の R の下で造られました 
3: パッケージ ‘lubridate’ はバージョン 4.4.1 の R の下で造られました 


CompletedProcess(args=['Rscript', '/Users/ogawaakane/Yugi_lab_project/phosphoproteome/scripts/R/kin_lineplot.R', '/Users/ogawaakane/Yugi_lab_project/phosphoproteome'], returncode=0)